<a href="https://colab.research.google.com/github/Diegoarmando24/Tarea-1/blob/main/Programas2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.linalg import lu

# Función para obtener una submatriz eliminando una fila y una columna
def SubMat(Mat, ren, col):
    """
    Elimina el renglón y la columna especificados de la matriz dada.
    """
    M1 = np.copy(Mat)
    M1 = np.delete(M1, ren, axis=0)  # Eliminar fila
    M1 = np.delete(M1, col, axis=1)  # Eliminar columna
    return M1

# Función para calcular el determinante de una matriz cuadrada de manera recursiva
def Det(Mat):
    """
    Calcula el determinante de una matriz cuadrada utilizando la expansión de cofactores.
    """
    if Mat.shape[0] == 2 and Mat.shape[1] == 2:
        return Mat[0][0] * Mat[1][1] - Mat[0][1] * Mat[1][0]

    deter = 0.0
    for col in range(Mat.shape[0]):
        deter += ((-1) ** col) * Mat[0][col] * Det(SubMat(Mat, 0, col))
    return deter

# Función para calcular la transpuesta de una matriz
def Transpuesta(Mat):
    """
    Calcula la transpuesta de una matriz cuadrada modificando la matriz original.
    """
    for ren in range(Mat.shape[0]):
        for col in range(Mat.shape[1]):
            if ren < col:
                Mat[ren, col], Mat[col, ren] = Mat[col, ren], Mat[ren, col]
    return Mat

# Función para calcular la matriz de cofactores
def Cofactores(Mat):
    """
    Calcula la matriz de cofactores de una matriz cuadrada.
    """
    Cofa = np.zeros_like(Mat, dtype=float)
    for ren in range(Mat.shape[0]):
        for col in range(Mat.shape[1]):
            Cofa[ren, col] = ((-1) ** (ren + col)) * Det(SubMat(Mat, ren, col))
    return Cofa

# Función para calcular la inversa de una matriz cuadrada
def Inv(Mat):
    """
    Calcula la inversa de una matriz cuadrada utilizando la matriz de cofactores.
    """
    deter = Det(Mat)
    if deter == 0:
        print("La matriz no tiene inversa.")
        return None

    Cofac = Cofactores(Mat)
    Cofac = Transpuesta(Cofac)
    Inversa = (1 / deter) * Cofac
    return Inversa

# Función para resolver un sistema usando la matriz inversa
def SolveInv(Mat, vec):
    """
    Resuelve un sistema de ecuaciones usando la inversa de la matriz de coeficientes.
    """
    InvMat = Inv(Mat)
    Solucion = InvMat @ vec
    return Solucion

# Sustitución hacia atrás
def SustitucionAtras(Mat, b):
    """
    Resuelve un sistema de ecuaciones con una matriz triangular superior.
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        SumCum = 0.0
        for j in range(i+1, n):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

# Sustitución hacia adelante
def SustitucionDelante(Mat, b):
    """
    Resuelve un sistema de ecuaciones con una matriz triangular inferior.
    """
    n = Mat.shape[0]
    x = np.zeros(n)
    for i in range(n):
        SumCum = 0.0
        for j in range(i):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]
    return x

# Resolución de sistemas con matrices diagonales
def SolverDiagonal(A, b):
    """
    Resuelve un sistema de ecuaciones donde la matriz es diagonal.
    """
    n = len(A)
    x = np.zeros_like(A, dtype=float)
    for i in range(n):
        x[i] = b[i] / A[i]
    return x

# Algoritmo de Thomas para matrices tridiagonales
def Thomas(DP, DS, DI, b):
    """
    Implementación del algoritmo de Thomas para resolver sistemas tridiagonales.
    """
    n = len(DP)
    x = np.zeros(n)
    for i in range(1, n):
        DP[i] -= (DI[i-1] / DP[i-1]) * DS[i-1]
    x[-1] = b[-1] / DP[-1]
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - DS[i] * x[i+1]) / DP[i]
    return x

# Medición de tiempos para cada método
def medir_tiempos():
    tamanos = list(range(3, 11))
    tiempos = {"Determinante": [], "Inversa": [], "Sustitución Atrás": [],
               "Sustitución Adelante": [], "Solver Diagonal": [], "Algoritmo de Thomas": []}

    for n in tamanos:
        A = np.array([[j + i*n for j in range(1, n+1)] for i in range(n)], dtype=float)
        b = np.array([i+1 for i in range(n)], dtype=float)
        D = np.diag([i+1 for i in range(n)])
        DP = 2 * np.ones(n)
        DS = -1 * np.ones(n-1)
        DI = -1 * np.ones(n-1)
        P, L, U = lu(A)
        for metodo, funcion in zip(tiempos.keys(), [Det, SolveInv, SustitucionAtras, SustitucionDelante, SolverDiagonal, Thomas]):
            inicio = time.time()
            funcion(A, b) if metodo in ["Inversa", "Sustitución Atrás", "Sustitución Adelante", "Solver Diagonal"] else funcion(A)
            tiempos[metodo].append(time.time() - inicio)
    return tamanos, tiempos

# Obtener tiempos y graficar resultados
tamanos, tiempos = medir_tiempos()
plt.figure(figsize=(10,6))
for metodo, valores in tiempos.items():
    plt.plot(tamanos, valores, label=metodo, marker='o', linestyle='--')
plt.xlabel("Tamaño de la Matriz")
plt.ylabel("Tiempo de Ejecución (s)")
plt.title("Comparación de Métodos Numéricos")
plt.legend()
plt.grid(True)
plt.show()
